In [21]:
import pandas as pd
import numpy as np
from gurobipy import GRB
import gurobipy as gp

In [22]:
investments = [
    [0, 4.1, 5.8, 6.5, 6.8],
    [0, 1.8, 3.0, 3.9, 4.5],
    [0, 1.5, 2.5, 3.3, 3.8],
    [0, 2.2, 3.8, 4.8, 5.5],
    [0, 1.3, 2.4, 3.2, 3.9],
    [0, 4.2, 5.9, 6.6, 6.8],
    [0, 2.2, 3.5, 4.2, 4.6],
    [0, 1.0, 1.7, 2.3, 2.8]
]

In [23]:
m = gp.Model("portfolio")

In [24]:
## Allocate decision variables
x = m.addMVar(shape=(len(investments), 5), vtype=GRB.BINARY, name="decision")

In [25]:
# ## Add constraints on max/min investment levels

for i in range(x.shape[0]):
    m.addConstr(
        0 <= gp.quicksum(x[i])
    )
    m.addConstr(
        gp.quicksum(x[i]) <= 1
    )


In [26]:
## Add constraint on total funding
m.addConstr(
    gp.quicksum(x[i][k] * 10 * k for i in range(x.shape[0]) for k in range(x.shape[1])) <= 100
)

<gurobi.Constr *Awaiting Model Update*>

In [27]:
## First part is revenue from investments, second part is return form savings
obj_f = gp.quicksum([investments[i][k] * x[i][k] for i in range(x.shape[0]) for k in range(x.shape[1])])\
 + (100 -  gp.quicksum(x[i][k] * 10 * k for i in range(x.shape[0]) for k in range(x.shape[1]))) * 0.05

In [28]:
m.setObjective(
    obj_f
    ,GRB.MAXIMIZE
)

In [29]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 17 rows, 40 columns and 112 nonzeros
Model fingerprint: 0xacbcbb4e
Variable types: 0 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e-01, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 19.3000000
Presolve removed 8 rows and 13 columns
Presolve time: 0.00s
Presolved: 9 rows, 27 columns, 54 nonzeros
Variable types: 0 continuous, 27 integer (27 binary)
Found heuristic solution: objective 20.7000000

Root relaxation: objective 2.230000e+01, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      22.3000000   22.30000  0.00%     -    

In [30]:
x.x

array([[ 0.,  0.,  1., -0.,  0.],
       [ 0.,  1.,  0., -0., -0.],
       [ 0.,  1., -0., -0.,  0.],
       [ 0., -0.,  1., -0., -0.],
       [ 0.,  1., -0., -0., -0.],
       [ 0.,  0.,  1., -0.,  0.],
       [ 0.,  1.,  0., -0.,  0.],
       [ 0., -0., -0., -0.,  0.]])